In [ ]:
import cv2
import time
from matplotlib import pyplot as plt
import numpy as np

test_frames = []
for i in range(20):
    frame = cv2.imread('test_frames\\frame_' + str(i) + '.png') 
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    test_frames.append(frame)
print(len(test_frames))

## Algorytm Fixed Threshold:

### Parametry wejściowe:
- resolution: rozdzielczość kamery CMOS lub obrazu wejściowego
- beta: próg wykrycia potencjalnego uderzenia cząsteczki
- gamma: próg na uśrednionej wartości sum pikselów ramki
- delta: procent czarnych pikseli ramki (pikesli o wartości 0)

### Przebieg algorytmu:
1. Wyłapanie ramki z sensora CMOS
2. Utworzenie pojedynczej ramki, gdzie każdemu pikselowi przyporządkowujemy sumę wartości kanałów RGB odpowiadającego piksela z oryginalnej ramki
3. Wyznaczenie maksimum z utworzonej ramki
4. Przelicznie ilości zer w utworzonej ramce
5. Wyznacznenie sumy wszystkich wartośći w ramce
6. Wyznaczenie wartości średniej pikseli dzieląc sumę wartości pikseli przez rozdzielczość ramki
7. Wyznaczenie procentu ilości zer w ramce dzieląc ich liczbę przez rozdzielczość ramki
8. Jeśli wcześniej wyliczone maksimum przekracza próg wykrycia potencjalnego uderzenia, średnia wartość pikseli przekracza próg na uśrednionej wartości i ilość czarnych pikseli przekracza próg na ułamku ilości czarnych algorytm zapisuje oryginalną ramkę

### Podsumowanie:
Algorytm wymaga analizy wcześniejszych danych, aby ustalić odpowiednie progi przed rozpoczęciem działania algorytmu, jednak pozwala to ograniczyć zużycie mocy obliczeniowej zużytej przez działanie algorytmu poprzez nie obliczanie progów podczas działania

In [ ]:
# frame:      input frame from CMOS camera
# resolution: CMOS image resolution
# beta:       threshold on potential hit detection
# gamma:      threshold on averaged Frame pixels value sum
# delta:      threshold on fraction of black pixels count

def alg1(frames: list[np.array], resolution: tuple[int ,int], beta, gamma, delta=0.04):
    potential_hits = []
    (Y, X) = resolution
    for frame in frames:
        r,g,b = cv2.split(frame.astype(float))
        frame_sum = r + g + b
        max_frame_sum = np.max(frame_sum)
        zero_count_frame_sum = 0
        for j in range(Y):
            for i in range(X):
                zero_count_frame_sum += 1 if frame_sum[j, i] == 0 else 0
        sum_frame_sum = frame_sum.sum()
        avg_frame_sum = sum_frame_sum / (Y*X)
        blacks_frame_sum = zero_count_frame_sum / (Y*X)
        if max_frame_sum > beta and avg_frame_sum < gamma and blacks_frame_sum > delta:
            potential_hits.append(frame)
    return potential_hits

## Algorytm Adaptive Threshold:

### Parametry wejściowe:
- resolution: rozdzielczość kamery CMOS lub obrazu wejściowego
- zone_siez: rozmiar blocków, na które podzielona zostanie ramka wejściowa

### Przebieg algorytmu:
1. Obliczamy na ile kwadratów zostanie podzielona ramka, dzieląca dwa pierwsze wymiary rozdzielczości przez rozmiar kwadratu
2. Inicjujemy zerami heatmape pikseli, tablice wyników pikseli i początkowy próg
3. Zaczynamy ciągłą pętle
   1. Wyłapanie ramki z sensora CMOS
   2. Utworzenie pojedynczej ramki, gdzie każdemu pikselowi przyporządkowujemy sumę wartości kanałów RGB odpowiadającego piksela z oryginalnej ramki
   3. Utworzenie tablicy różnic pomiędzy utworzoną ramką a heatmapą 
   4. W pętli przechodzącej po wszystkich pikselach zsumowanej ramki każdemu pikselowi dla którego różnica przekroczyła 4-krotną wartość w hetmapie przpisujemy kwadrat jego różnicy, w przeciwnym razie zero; aktualizujemy heatmapę przypisując pikselom kombinację liniową ich starych i nowych wartości
   5. W pętli przechodzącej po wszyskich wartościach w tablicy wyników pikesli dokonujemy ich agregacji do bloków i uzyskane w ten sposób wartości zapisujemy w zainicjowanej tablicy bloków o zredukowanych wymiarach
   6. Wyliczamy próg, inicjalizowany drugą najwyższą wartością w tablicy bloków, wyliczany jak kombinacja obecnej drugiej najwyższej wartości i swojej poprzedniej wartości
   7. W pęli przechodzącej po wszystkich wartościach tablicy bloków, jeśli wyliczony próg zostanie 2-krotnie przekroczony algorytm zapisuje orginalną ramkę jako potencjalne uderzenie

### Podsumowanie:
Algorytm nie wymaga wcześniejszej analizy danych w celu wyznaczenia progów, jednak wykonuje znacznie więcej obliczeń niż pierwszy algorytm

In [ ]:
# resolution: CMOS image resolution
# zone_size:  size of square block on which input frame will be divided

def alg2(frames: list[np.array], resolution: tuple[int ,int], zone_size):
    potential_hits = []
    trigger_map_width = resolution[1] // zone_size
    trigger_map_height = resolution[0] // zone_size
    heat = np.zeros(resolution)
    pixel_scores = np.zeros(resolution)
    threshold = 0
    for frame in frames:
        r,g,b = cv2.split(frame.astype(float))
        total = r + g + b

        diff = total - heat

        for b in range(resolution[0]):
            for a in range(resolution[1]):
                if total[b, a] > 4 and diff[b, a] > 4 * heat[b, a]:
                    pixel_scores[b, a] = diff[b, a] * diff[b, a]
                else:
                    pixel_scores[b, a] = 0
                heat[b, a] = 0.03 * total[b, a] + (1 - 0.03) * heat[b, a]

        block_scores = np.zeros((trigger_map_height, trigger_map_width))
        for b in range(resolution[0]):
            for a in range(resolution[1]):
                block_scores[b // zone_size, a // zone_size] += pixel_scores[b, a]

        un = np.sort(block_scores, axis=None)
        max2_nd = un[len(un) - 2]
        
        if threshold == 0:
            un = np.sort(block_scores, axis=None)
            threshold = max2_nd
        else:
            threshold = 0.1 * max2_nd + (1 - 0.1) * threshold
        
        flag = False
        for b in range(1, trigger_map_height - 1):
            for a in range(1, trigger_map_width - 1):
                if block_scores[b, a] > 2 * threshold:
                    potential_hits.append(frame)
                    flag = True
                    break
            if flag:
                break
    return potential_hits

## Algorytm Adaptive Threshold Low Power Consumption Version

### Parametry wejściowe:
- resolution: rozdzielczość obrazu wejściowego lub kamery CMOS
- edge: szerkokość krawędzi obrazu, która zostanie obcięta podczas procesu
- kernel: jądro macierzy gaussa potrzebne do wykonania szumu gaussa
- imc: liczba wstępnych iteracji wykorzystanych do wylicznia ruchomej średniej
- alfa: współczynnik pamięci algorytmu
- theta: próg potencjalnej detekcji

### Przebieg Algorytmu:
1. Inicjalizacja Ramki, która jest ruchomą średnią tak aby była pusta
2. Wyłapanie ramki z sensora CMOS
3. Utworzenie pojedynczej ramki, gdzie każdemu pikselowi przyporządkowujemy sumę wartości kanałów RGB odpowiadającego piksela z oryginalnej ramki
4. Obcięcie krawędzi
5. Przeprowadzenie operacji MaxPooling ze skalą, co powoduje zmniejszenie rozdzielczości o 2 w każdym wymiarze, na ramce
6. Dodanie szumu gaussowskiego do ramki po MaxPoolingu
7. Jeśli imc jest obecnie mniejsze lub równe zeru, algorytm przechodzi przez wszystkie piksele w celu ustalenia czy w ramce nie znajduje się potencjalne uderzenie. Warunkiem zapisu jest jeśli czterokrotność odpowiednika piksela w ruchomej średniej jest mniejsza od jego odpowiednika w utworzonej ramce z szumem gaussowskim i jego odpowiednik w ramce przed dodaniem szumu jest większy od progu potencjalnego trafienia.
8. Jeśli imc jest większe od 0, dekrementujemy je o 1
9. Jeśli ruchoma średnia jest pusta, ruchomej średniej przypisywana jest wartość ramki po dodaniu szumu. W przeciwnym wypadku, ruchomej średniej jest przypisywana suma obecnej ruchomej średniej pomnożonej przez pamięć ruchomej średniej minus 1 i ramki z szumem pomnożonej przez pamięć ruchomej średniej

### Podusmowanie
Ta wersja algorytmu ogranicza zużycie pamięci obliczeniowej poprzez użycie jednego obiektu w celu ustalenia jednego z kryteriów wykrycia trafienia i przez zmniejszenie ilość obliczeń, jak i przez 

In [ ]:
def max_pooling(frame, num):
    rows = frame.shape[0] // num
    cols = frame.shape[1] // num
    frame_pool = np.zeros((rows, cols))
    
    for j in range(rows):
        for i in range(cols):
            frame_pool[j, i] = np.max(frame[j * num: j * num + num, i * num: i * num + num])
    return frame_pool

# edge:       width of image edge in pixel which will be cropped during processing
# kernel:     gaussian kernel matrix
# imc:        number of initial iterations devoted to initialise moving average
# resolution: CMOS image resolution
# alfa:       moving average memory
# theta:      threshold of potential hit detections

def alg3(frames: list[np.array], resolution: tuple[int ,int], edge, kernel, imc, alfa, theta):
    potential_hits = []
    frame_avg = None
    for frame in frames:
        r, g, b = cv2.split(frame.astype(float))
        frame_sum = r + g + b
        frame_crop = frame_sum[edge:resolution[0] - edge, edge:resolution[1] - edge] 
        frame_pool = max_pooling(frame_crop, 2)
        frame_gaussian = cv2.GaussianBlur(frame_pool, kernel, 0)
        
        flag = False
        if imc <= 0:
            for b in range(frame_gaussian.shape[0]):
                for a in range(frame_gaussian.shape[1]):
                    if frame_avg[b, a] * 4 < frame_gaussian[b, a] and frame_pool[b, a] > theta:
                        potential_hits.append(frame)
                        flag = True
                        break
                if flag:
                    break
        else:
            imc -= 1

        if frame_avg is None:
            frame_avg = frame_gaussian
        else:
            frame_avg = (alfa - 1) * frame_avg + alfa * frame_gaussian
    return potential_hits

In [ ]:
start = time.time()
hits_alg1 = alg1(test_frames, (1080, 1920), 120, 40)
end = time.time()
time1 = end - start

start = time.time()
hits_alg2 = alg2(test_frames, (1080, 1920), 20)
end = time.time()
time2 = end - start

start = time.time()
hits_alg3 = alg3(test_frames, (1080, 1920), 30, (3, 3), 12, 0.2, 255)
end = time.time()
time3 = end - start

In [ ]:
print(f"Algorithm 1. detected {len(hits_alg1)}. possible hits. Runtime: {time1}")
print(f"Algorithm 2. detected {len(hits_alg2)}. possible hits. Runtime: {time2}")
print(f"Algorithm 3. detected {len(hits_alg3)}. possible hits. Runtime: {time3}")

In [ ]:
plt.imshow(test_frames[0], cmap='gray')

In [ ]:
plt.imshow(test_frames[17], cmap='gray')